import the necessary libraries

In [55]:
import numpy as np
import pandas as pd
import re
import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


Load the dataset into your preferred programming environment.

In [56]:
df=pd.read_csv("/content/tweet_emotions .csv")

Display the first 10 rows of the dataset

In [57]:
df.head(10)

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?


Preprocessing

In [58]:
df.shape

(40000, 3)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [60]:
df.columns

Index(['tweet_id', 'sentiment', 'content'], dtype='object')

In [61]:
df.isna().sum()

tweet_id     0
sentiment    0
content      0
dtype: int64

In [62]:
df['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [63]:
df['content'][0]

'@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [64]:
df['content'][1]

'Layin n bed with a headache  ughhhh...waitin on your call...'

In [65]:
df['content'][3]

'wants to hang out with friends SOON!'

In [66]:
df['content'][15]

'The storm is here and the electricity is gone'

In [67]:
df['content'][20000]

'Thanks for pointing out the crucial problems @thakkar. Both of them have been taken care of  (cc: @Netra)'

In [68]:
df['content'][39999]

'@mopedronin bullet train from tokyo    the gf and i have been visiting japan since thursday  vacation/sightseeing    gaijin godzilla'

Preprocess the data by removing any unnecessary columns and cleaning the text data

In [69]:
# Drop the 'id' column
df = df.drop("tweet_id", axis=1)


In [70]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import re
import unidecode

In [72]:
def clean_keywords(word):
    return re.sub(r'%20', ' ', word)
def remove_accents(word):
    return unidecode.unidecode(word)
def remove_punctuation(word):
    return re.sub(r"[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n -' ]"," ",word)
def cleaning_URLs(word):
    return re.sub('((www.[^s]+)|(https?:\/\/.*?[\s+]))',' ',word)
def remove_mentions(word):
    return re.sub('@[\w]*',' ',word)
def to_lowercase(word):
    return word.lower()


In [73]:
df['content'] = df['content'].apply(lambda x: to_lowercase(x))
df['content'] = df['content'].apply(lambda x: cleaning_URLs(x))
df['content'] = df['content'].apply(lambda x: remove_mentions(x))
df['content'] = df['content'].apply(lambda x: remove_accents(x))
df['content'] = df['content'].apply(lambda x: remove_punctuation(x))
df['content'] = df['content'].apply(lambda x: clean_keywords(x))


In [74]:
df

,sentiment,content
0,empty,i know i was listenin to bad habit earlier ...
1,sadness,layin n bed with a headache ughhhh waitin o...
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,wants to hang out with friends soon
4,neutral,we want to trade with someone who has housto...
...,...,...
39995,neutral,
39996,love,happy mothers day all my love
39997,love,happy mother s day to all the mommies out ther...
39998,happiness,wassup beautiful follow me peep out my...


In [75]:
df['content'][39999]

'  bullet train from tokyo    the gf and i have been visiting japan since thursday  vacation sightseeing    gaijin godzilla'

Create a dictionary to map sentiment values to numerical labels

In [76]:
sentiment_mapping = {
    'neutral': 0,
    'worry': 1,
    'happiness': 2,
    'sadness': 3,
    'love': 4,
    'surprise': 5,
    'fun': 6,
    'relief': 7,
    'hate': 8,
    'empty': 9,
    'enthusiasm': 10,
    'boredom': 11,
    'anger': 12
}

# Map the sentiment values to numerical labels
df['sentiment'] = df['sentiment'].map(sentiment_mapping)



In [77]:
df

,sentiment,content
0,9,i know i was listenin to bad habit earlier ...
1,3,layin n bed with a headache ughhhh waitin o...
2,3,funeral ceremony gloomy friday
3,10,wants to hang out with friends soon
4,0,we want to trade with someone who has housto...
...,...,...
39995,0,
39996,4,happy mothers day all my love
39997,4,happy mother s day to all the mommies out ther...
39998,2,wassup beautiful follow me peep out my...


 Split the data into training and testing sets

In [78]:

X = df["content"]
y = df["sentiment"]

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Using Logistic regression machine learning algorithm to train a sentiment analysis model on the training set.

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:

log= Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logr', LogisticRegression()),
])


In [82]:
log.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('logr', LogisticRegression())])

Evaluating the model

In [83]:
y_pred_log = log.predict(X_test)
print("accuracy score :",accuracy_score(y_test, y_pred_log))


accuracy score : 0.344625


Predicting the model using sample input

 'neutral': 0,
    'worry': 1,
    'happiness': 2,
    'sadness': 3,
    'love': 4,
    'surprise': 5,
    'fun': 6,
    'relief': 7,
    'hate': 8,
    'empty': 9,
    'enthusiasm': 10,
    'boredom': 11,
    'anger': 12

In [84]:
# Define a sample tweet to predict its sentiment
tweet = "surprise,Got the news"
prediction = log.predict([tweet])
print("The sentiment of the tweet is:", prediction[0])

The sentiment of the tweet is: 5


In [85]:
tweet = "sadness,So sleepy again and it's not even that late. I fail once again."
prediction = log.predict([tweet])
print("The sentiment of the tweet is:", prediction[0])

The sentiment of the tweet is: 3


In [86]:
tweet="hate,@FlittyGadabout I'm not a hobo."
prediction = log.predict([tweet])
print("The sentiment of the tweet is:", prediction[0])

The sentiment of the tweet is: 8
